# Early Stopping if the Loss hits Minimum

In [1]:
from twilio.rest import Client

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.datasets import make_classification

import tensorflow as tf
from tensorflow import keras 

import numpy as np

D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
def send_message(text):
    account_sid = 'AC627ada27fffdce01d175164bcba0d23c' #Can be obtained from Twilio Console
    auth_token = '729c156bf50bc7bc1fb49c2256cd9836'  #Can be obtained from Twilio Console
    client = Client(account_sid, auth_token)

    message = client.messages \
    .create(
         from_='whatsapp:+14155238886',
         body=text,
         to='whatsapp:+917400452552'
     )

In [3]:
class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                send_message("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            send_message("Epoch %03d: early stopping" % (self.stopped_epoch + 1))

In [4]:
cb = EarlyStoppingAtMinLoss() #Creating the Callback

In [5]:
x,y = make_classification(n_samples=1000, n_classes=2, n_features=20)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [6]:
def create_model():
    model = Sequential()
    model.add(Dense(60, input_dim=20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.RMSprop(learning_rate=0.1), metrics=["mean_absolute_error"])

    return model

model = create_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
model.fit(x_train,y_train,batch_size=32, epochs=20, callbacks=[cb], verbose=1) #Fitting the model

Epoch 1/20
700/700 [==============================] - 0s 135us/sample - loss: 0.1922 - mean_absolute_error: 0.2195
Epoch 2/20
700/700 [==============================] - 0s 43us/sample - loss: 0.1753 - mean_absolute_error: 0.1855
Epoch 3/20
700/700 [==============================] - 0s 33us/sample - loss: 0.1334 - mean_absolute_error: 0.1391
Epoch 4/20
700/700 [==============================] - 1s 2ms/sample - loss: 0.1525 - mean_absolute_error: 0.1554
